In [11]:
import pandas as pd
import seaborn as sns
import numpy as np

In [12]:
df = pd.read_csv("data/acled.csv")

In [48]:
#df.columns
df.tags.value_counts()
#df.admin1.value_counts()
#df.admin3.value_counts()

tags
crowd size=no report                               675702
crowd size=hundreds                                 26977
crowd size=dozens                                   23241
crowd size=large                                    12377
local administrators                                12119
                                                    ...  
counter-demonstration; crowd size=at least 230          1
counter-demonstration; crowd size=about 127-132         1
crowd size=between 1,600 and 1,640                      1
crowd size=at least 2,100                               1
crowd size=at least 35; sexual violence                 1
Name: count, Length: 13906, dtype: int64

In [54]:

##crowdsize
df["crowdsize"] = df['tags'].str.replace(r'(?i)^.*crowd size', 'crowd size', regex=True)
df["crowdsize"] = df["crowdsize"].str.replace(r';.*', '', regex=True)
value_counts_crowdsize = df.crowdsize.value_counts()
ones_count_crowdsize = value_counts_crowdsize[value_counts_crowdsize == 1]
ones_count_crowdsize

crowdsize
crowd size=modest group                 1
crowd size=nearly 40000                 1
crowd size=approximately 12-15          1
crowd size=about 20-60                  1
crowd size=approximately 56 drivers     1
                                       ..
crowd size=between 16,000 and 50,000    1
crowd size=between 7,200 and 15,000     1
crowd size=between 4,900 and 8,000      1
crowd size=between 25,000 and 55,000    1
crowd size=100000                       1
Name: count, Length: 7529, dtype: int64

In [55]:
##word before number 
pattern = r'([a-zA-Z\s]+)(?=\s*\d)'
df["words before number"] = df["crowdsize"].str.extract(pattern)
df["words before number"] = df["words before number"].str.replace(r'^\s+', '', regex=True)
value_counts_wordsbeforenumber = df["words before number"].value_counts()
value_counts_wordsbeforenumber


words before number
around                        54227
about                         36195
more than                     15293
at least                       8684
over                           7149
                              ...  
a couple dozen to roughly         1
approaching                       1
tractors and some                 1
betwen                            1
several hundred to about          1
Name: count, Length: 446, dtype: int64

In [56]:
##key
df["keys"] = df["tags"].str.extract(r'^(.*?)(?=;)')
df["keys"].value_counts()


keys
crowd size=no report              3769
counter-demonstration             2757
armed                              786
schools                            451
local administrators               398
                                  ... 
crowd size=around half a dozen       1
crowd size=about 125                 1
crowd size=over 60                   1
crowd size=around 14,000             1
crowd size=at least 35               1
Name: count, Length: 463, dtype: int64

In [58]:
##values
df["values"] = df['tags'].str.extract(r'(?<=;)(.*)')
df["values"].value_counts()


values
local administrators                                                              3616
crowd size=no report                                                              1550
sexual violence                                                                    527
stop the steal                                                                     357
women targeted: accused of witchcraft/sorcery                                      345
                                                                                  ... 
crowd size=less than 100                                                             1
crowd size=around 200; stop the steal                                                1
crowd size=around three dozen                                                        1
crowd size=nearly 56                                                                 1
crowd size=12; women targeted: activists/human rights defenders/social leaders       1
Name: count, Length: 1088, dtype: in